In [1]:
from netCDF4 import Dataset
import cartopy.crs as ccrs
import matplotlib.animation as animation
from IPython import display
import cartopy
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd

In [2]:
d = Dataset('/media/robbie/TOSHIBA EXT/E5_ros/2020.nc')
d

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2021-05-20 14:03:18 GMT by grib_to_netcdf-2.16.0: /opt/ecmwf/eccodes/bin/grib_to_netcdf -S param -o /cache/data9/adaptor.mars.internal-1621517556.0058398-32293-1-85e00015-28c4-40d3-a45f-632335e0b93a.nc /cache/tmp/85e00015-28c4-40d3-a45f-632335e0b93a-adaptor.mars.internal-1621517556.0079916-32293-1-tmp.grib
    dimensions(sizes): longitude(360), latitude(31), time(8784)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 ptype(time,latitude,longitude), int16 sf(time,latitude,longitude), int16 tp(time,latitude,longitude), int16 t2m(time,latitude,longitude)
    groups: 

In [3]:
lon = d['longitude']
lat = d['latitude']

In [4]:
type_data_raw = np.array(d['ptype'])
prec_data_raw = np.array(d['tp'])*1000

In [17]:
%matplotlib widget

type_data = type_data_raw.copy()
prec_data = prec_data_raw.copy()

fig = plt.figure()    

ax = plt.axes(projection=ccrs.NorthPolarStereo())


ax.add_feature(cartopy.feature.LAND, edgecolor='black',zorder=1)

ax.set_extent([-180, 180, 90, 66], ccrs.PlateCarree())

bg = ax.pcolormesh(np.array(lon), np.array(lat), np.full(type_data[0].shape, np.nan),
                   vmin = 0, vmax = 9,
             transform=ccrs.PlateCarree(),zorder=0,cmap=my_cmap)

anno = ax.annotate('',xycoords='axes fraction',xy=(0.99,0.93),ha='right',fontsize='x-large')

ax.scatter(101.184955,89.005759,transform=ccrs.PlateCarree(),s=100,color='r',marker='x')

type_data[prec_data<(5/24)] = 0

cbar = fig.colorbar(bg,ax=ax,ticks=np.arange(-0.5,8.51,1))

cbar.ax.set_yticklabels(['']+get_grain_tick_labels())

# def animate(frame):

#     bg.set_array(np.rint(type_data[frame])[:-1,:-1].ravel())
    
#     doy = frame/24
    
#     date = datetime.date(2020,1,1) + datetime.timedelta(days=doy)
    
#     anno.set_text(f'{date}')


# #######################################################

# ani = animation.FuncAnimation(fig,
#                               animate,
# #                              frames= np.arange(8700,8784,24),
#                              frames= np.arange(0,8784,24),
#                              )    

# video = ani.to_html5_video()
# html = display.HTML(video)
# display.display(html)
# plt.close()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(1, -0.5, ''),
 Text(1, 0.5, 'No Precipitation'),
 Text(1, 1.5, 'Rain'),
 Text(1, 2.5, 'NA'),
 Text(1, 3.5, 'Freezing rain'),
 Text(1, 4.5, 'NA'),
 Text(1, 5.5, 'Snow'),
 Text(1, 6.5, 'Wet snow'),
 Text(1, 7.5, 'Rain/Snow Mix'),
 Text(1, 8.5, 'Ice pellets')]

In [18]:
type_data[prec_data<(5/24)] = 0

start_doy = datetime.datetime(2020,9,10).timetuple().tm_yday
end_doy = datetime.datetime(2020,9,17).timetuple().tm_yday

ani = animation.FuncAnimation(fig,
                              animate,
                             frames= range((start_doy-1)*24,(end_doy-1)*24,1),
                             )    

video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

# Find nearest reanalysis pixel to PS on 14th

In [11]:
datetimes = [datetime.datetime(2020,1,1)+datetime.timedelta(hours=x) for x in range(8784)]

In [19]:
101.184955,89.005759

lon_ps, lat_ps = np.argmin(abs(np.array(lon)-101.185)), np.argmin(abs(np.array(lat)-89.00))

fig, ax = plt.subplots(1,1)
ax.plot(datetimes,prec_data[:,lat_ps,lon_ps])
ax.set_ylabel('Precipitation (mm/hr)')
ax.set_xlim(datetime.datetime(2020,9,12),datetime.datetime(2020,9,15))
ax.tick_params('x',rotation=45)

ax2 = ax.twinx()

type_series = type_data[:,lat_ps,lon_ps].copy()

type_series[type_series==0] = np.nan

ax2.scatter(datetimes,type_series,color='k',marker='.')

ax2.set_yticks(np.arange(0,9))
ax2.set_yticklabels(get_grain_tick_labels())

plt.tight_layout()

plt.savefig('ROS_sept_2020.jpg',dpi=500,bbox_inches='tight')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
from matplotlib.colors import LinearSegmentedColormap

def grain_type_colormap():

    """Returns a matplotlib colormap suitable for grain-type classification plots"""

    colors = [('white'),  # 1, precipitation particles
              ('green'),  # 2, Decomposing fragmented PP
              ('yellow'),  # 3 Rounded Grains (RG)
              ('orange'),  # 4 Faceted Crystals
              ('blue'),  # 5 Depth Hoar
              ('gray'),  # 6 Surface Hoar
              ('red'),  # 7 Melt FOrms
              ('cyan'),  # 8 Ice formations
              ('black'),  # 9 Rounded faceted particles
              ]


    cmap_name = 'grain_type_map'

    my_cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=9)

    return(my_cmap)

def get_grain_tick_labels():

    """Returns a list of labels for a grain-type classification plot"""

    grain_tick_labels = ['No Precipitation',
                         'Rain',
                         'NA',
                         'Freezing rain',
                         'NA',
                         'Snow',
                         'Wet snow',
                         'Rain/Snow Mix',
                         'Ice pellets']

    return(grain_tick_labels)

my_cmap = grain_type_colormap()